# Importing libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
import os
import math as mt
from astropy.modeling import models
from astropy import units as u
from scipy.optimize import curve_fit
from scipy.optimize import newton, least_squares
from astropy.timeseries import LombScargle
from scipy.special import voigt_profile
import Analysis_functions as AF
from importlib import reload
import scipy.stats as ss
import matplotlib.colors as mc
import itertools
import csv
import pandas as pd
from scipy.interpolate import interp1d
import bindensity as bind
import itertools
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
import spectrum_model as spec_mod
import line_model as line_mod

# Preliminary info.

In [ ]:
days = ['2023-01-20', '2023-01-21', '2023-01-22', '2023-01-23', '2023-01-24', '2023-01-25', '2023-01-26', 
        '2023-01-27', '2023-02-01', '2023-02-02', '2023-02-03', '2023-02-06', '2023-02-15', '2023-02-16']

new = True
main_directory = '/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/new_Solar-data'

#Defining directories
blaze_dir = 'Blaze-data'
telluric_dir = 'Telluric-data'
dir_name = 'Corr-data'
CCF_dir_name = 'CCF_RV-data'
rassine_dir_name = 'Big-Exclusion-Rassine-data'
Rassine = True

save_location = '/Users/samsonmercier/Downloads'
#Continuum Ranges used for normalization and EW calculation
normalization_poly_order = 0
fit_poly_order = 0
wav_ranges = [(10804.8, 10805.9), (10807.4, 10808.5), (10825.5, 10826.5), (10839, 10840), (10842.4, 10843.1), (10850.2, 10851.2), (10856.5, 10857.9)]

#Initializing dictionaries
#He triplet fitted parameters dictionnary
fit_params_dict = {}
fit_err_dict = {}

#Rassine data used
rassine_norm_spctr_dict = {}
rassine_norm_err_dict = {}
rassine_wav_dict = {}
mode_dict = {}
date_dict = {}

# Extracting and Reducing data

In [ ]:
for day in days:
    print('STARTING ',day)
    print('\n')
    location = main_directory + '/' + day
    #Extracting data
    save = False
    
    if new:
        print('New Extraction')
        if Rassine:
            print('Rassine')
            if os.path.exists(location+'/'+rassine_dir_name+'-HA'):
                print('Multiple modes')
                tot_lamda, rassine_total_lamda_HA, rassine_total_lamda_HE, rassine_master_lamda_HA, rassine_master_lamda_HE, tot_spctr, tot_norm_spctr, rassine_total_norm_spctr_HA, rassine_total_norm_spctr_HE, rassine_master_spctr_HA, rassine_master_spctr_HE, rassine_continuum_HA, rassine_continuum_HE, tot_err, tot_norm_err, tot_SNR, tot_mode, tot_date, tot_RV, tot_RV_err, tot_FWHM, tot_FWHM_err, tot_BIS_SPAN, tot_BIS_SPAN_err, tot_CONTRAST, tot_CONTRAST_err, tot_H2O, tot_H2O_err, tot_O2, tot_O2_err, tot_CO2, tot_CO2_err, tot_AIRM, tot_telluric = AF.new_extraction(location, dir_name, blaze_dir, CCF_dir_name, telluric_dir, rassine_dir_name, 14, wav_ranges, normalization_poly_order, save_location, Rassine, save)
            else:
                print('Single mode')
                tot_lamda, rassine_total_lamda, rassine_master_lamda, tot_spctr, tot_norm_spctr, rassine_total_norm_spctr, rassine_master_spctr, rassine_continuum, tot_err, tot_norm_err, tot_SNR, tot_mode, tot_date, tot_RV, tot_RV_err, tot_FWHM, tot_FWHM_err, tot_BIS_SPAN, tot_BIS_SPAN_err, tot_CONTRAST, tot_CONTRAST_err, tot_H2O, tot_H2O_err, tot_O2, tot_O2_err, tot_CO2, tot_CO2_err, tot_AIRM, tot_telluric = AF.new_extraction(location, dir_name, blaze_dir, CCF_dir_name, telluric_dir, rassine_dir_name, 14, wav_ranges, normalization_poly_order, save_location, Rassine, save)
        else:
            print('No Rassine')
            tot_lamda, tot_spctr, tot_norm_spctr, tot_err, tot_norm_err, tot_SNR, tot_mode, tot_date, tot_RV, tot_RV_err, tot_FWHM, tot_FWHM_err, tot_BIS_SPAN, tot_BIS_SPAN_err, tot_CONTRAST, tot_CONTRAST_err, tot_H2O, tot_H2O_err, tot_O2, tot_O2_err, tot_CO2, tot_CO2_err, tot_AIRM, tot_telluric = AF.new_extraction(location, dir_name, blaze_dir, CCF_dir_name, telluric_dir, rassine_dir_name, 14, wav_ranges, normalization_poly_order, save_location, Rassine, save)
    else:
        print('Old Extraction')
        tot_lamda, tot_norm_spctr, tot_norm_err, tot_SNR, tot_mode, tot_date, tot_RV, tot_RV_err, tot_FWHM, tot_FWHM_err, tot_BIS_SPAN, tot_BIS_SPAN_err, tot_CONTRAST, tot_CONTRAST_err, tot_H2O, tot_H2O_err, tot_O2, tot_O2_err, tot_CO2, tot_CO2_err, tot_AIRM = AF.extraction(dir_name, blaze_dir, CCF_dir_name, 14)

    #Reducing the data
    print('REDUCING', '\n')
    #SNR and RV clipping
    print('SNR AND RV CLIPPING', '\n')
    sig = 200
    #Distinguish two cases depending on the number of modes of observation
    #If there are two modes of observation
    if np.sum(tot_mode=='A') != len(tot_mode) and np.sum(tot_mode=='E')!= len(tot_mode):
        total_SNR_HA, total_SNR_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_SNR, tot_RV, tot_AIRM, sig)

        #Special ones - BEGIN
        tota_norm_spctr_HA, tota_norm_spctr_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_norm_spctr, tot_RV, tot_AIRM, sig)

        tota_lamda_HA, tota_lamda_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_lamda, tot_RV, tot_AIRM, sig)

        tota_norm_err_HA, tota_norm_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_norm_err, tot_RV, tot_AIRM, sig)

        tota_spctr_HA, tota_spctr_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_spctr, tot_RV, tot_AIRM, sig)

        tota_err_HA, tota_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_err, tot_RV, tot_AIRM, sig)

        tota_telluric_HA, tota_telluric_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_telluric, tot_RV, tot_AIRM, sig)
        #Special ones - END

        date_HA, date_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_date, tot_RV, tot_AIRM, sig)

        total_RV_HA, total_RV_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_RV, tot_RV, tot_AIRM, sig)

        total_RV_err_HA, total_RV_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_RV_err, tot_RV, tot_AIRM, sig)

        total_FWHM_HA, total_FWHM_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_FWHM, tot_RV, tot_AIRM, sig)

        total_FWHM_err_HA, total_FWHM_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_FWHM_err, tot_RV, tot_AIRM, sig)

        total_BIS_SPAN_HA, total_BIS_SPAN_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_BIS_SPAN, tot_RV, tot_AIRM, sig)

        total_BIS_SPAN_err_HA, total_BIS_SPAN_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_BIS_SPAN_err, tot_RV, tot_AIRM, sig)

        total_CONTRAST_HA, total_CONTRAST_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CONTRAST, tot_RV, tot_AIRM, sig)

        total_CONTRAST_err_HA, total_CONTRAST_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CONTRAST_err, tot_RV, tot_AIRM, sig)

        total_H2O_HA, total_H2O_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_H2O, tot_RV, tot_AIRM, sig)

        total_H2O_err_HA, total_H2O_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_H2O_err, tot_RV, tot_AIRM, sig)

        total_O2_HA, total_O2_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_O2, tot_RV, tot_AIRM, sig)

        total_O2_err_HA, total_O2_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_O2_err, tot_RV, tot_AIRM, sig)

        total_CO2_HA, total_CO2_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CO2, tot_RV, tot_AIRM, sig)

        total_CO2_err_HA, total_CO2_err_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CO2_err, tot_RV, tot_AIRM, sig)

        total_AIRM_HA, total_AIRM_HE = AF.segment_and_reduce(tot_mode, tot_SNR, tot_AIRM, tot_RV, tot_AIRM, sig)

    #If there is one mode of observation
    else:
        #We don't need to segment and reduce the RASSINE spectra because they were made from 
        #spectra that were previously segmented and reduced. 

        #Special ones - BEGIN
        tota_norm_spctr = AF.segment_and_reduce(tot_mode, tot_SNR, tot_norm_spctr, tot_RV, tot_AIRM, sig)

        tota_lamda = AF.segment_and_reduce(tot_mode, tot_SNR, tot_lamda, tot_RV, tot_AIRM, sig)

        tota_norm_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_norm_err, tot_RV, tot_AIRM, sig)

        tota_spctr = AF.segment_and_reduce(tot_mode, tot_SNR, tot_spctr, tot_RV, tot_AIRM, sig)

        tota_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_err, tot_RV, tot_AIRM, sig)

        tota_telluric = AF.segment_and_reduce(tot_mode, tot_SNR, tot_telluric, tot_RV, tot_AIRM, sig)
        #Special ones - END

        total_SNR = AF.segment_and_reduce(tot_mode, tot_SNR, tot_SNR, tot_RV, tot_AIRM, sig)

        date = AF.segment_and_reduce(tot_mode, tot_SNR, tot_date, tot_RV, tot_AIRM, sig)

        total_RV = AF.segment_and_reduce(tot_mode, tot_SNR, tot_RV, tot_RV, tot_AIRM, sig)

        total_RV_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_RV_err, tot_RV, tot_AIRM, sig)

        total_FWHM = AF.segment_and_reduce(tot_mode, tot_SNR, tot_FWHM, tot_RV, tot_AIRM, sig)

        total_FWHM_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_FWHM_err, tot_RV, tot_AIRM, sig)

        total_BIS_SPAN = AF.segment_and_reduce(tot_mode, tot_SNR, tot_BIS_SPAN, tot_RV, tot_AIRM, sig)

        total_BIS_SPAN_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_BIS_SPAN_err, tot_RV, tot_AIRM, sig)

        total_CONTRAST = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CONTRAST, tot_RV, tot_AIRM, sig)

        total_CONTRAST_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CONTRAST_err, tot_RV, tot_AIRM, sig)

        total_H2O = AF.segment_and_reduce(tot_mode, tot_SNR, tot_H2O, tot_RV, tot_AIRM, sig)

        total_H2O_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_H2O_err, tot_RV, tot_AIRM, sig)

        total_O2 = AF.segment_and_reduce(tot_mode, tot_SNR, tot_O2, tot_RV, tot_AIRM, sig)

        total_O2_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_O2_err, tot_RV, tot_AIRM, sig)

        total_CO2 = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CO2, tot_RV, tot_AIRM, sig)

        total_CO2_err = AF.segment_and_reduce(tot_mode, tot_SNR, tot_CO2_err, tot_RV, tot_AIRM, sig)

        total_AIRM = AF.segment_and_reduce(tot_mode, tot_SNR, tot_AIRM, tot_RV, tot_AIRM, sig)

    mode = tot_mode
        
    #Making the Rassine error bars
    print('MAKING ERRORBARS', '\n')
    if np.sum(tot_mode=='A') != len(tot_mode) and np.sum(tot_mode=='E')!= len(tot_mode):
        #HA
        #Define the wavelength solution onto which we want to interpolate the error bars
        rassine_wav_sol_HA = np.append(rassine_total_lamda_HA[0], rassine_total_lamda_HA[0][-1] + np.diff(rassine_total_lamda_HA[0])[-1])

        #Initialize the Rassine error bars
        rassine_total_norm_err_HA = np.zeros(rassine_total_norm_spctr_HA.shape)

        for i in range(rassine_total_norm_spctr_HA.shape[0]):
            #Define the edges of the wavelength solution.
            base_wav_sol_HA = np.append(tota_lamda_HA[i], tota_lamda_HA[i][-1] + np.diff(tota_lamda_HA[i])[-1])
            rassine_total_norm_err_HA[i] = bind.resampling(rassine_wav_sol_HA, base_wav_sol_HA, tota_err_HA[i], kind='cubic')/rassine_continuum_HA[i]

        #HE
        #Define the wavelength solution onto which we want to interpolate the error bars
        rassine_wav_sol_HE = np.append(rassine_total_lamda_HE[0], rassine_total_lamda_HE[0][-1] + np.diff(rassine_total_lamda_HE[0])[-1])

        #Initialize the Rassine error bars
        rassine_total_norm_err_HE = np.zeros(rassine_total_norm_spctr_HE.shape)

        for i in range(rassine_total_norm_spctr_HE.shape[0]):
            #Define the edges of the wavelength solution.
            base_wav_sol_HE = np.append(tota_lamda_HE[i], tota_lamda_HE[i][-1] + np.diff(tota_lamda_HE[i])[-1])
            rassine_total_norm_err_HE[i] = bind.resampling(rassine_wav_sol_HE, base_wav_sol_HE, tota_err_HE[i], kind='cubic')/rassine_continuum_HE[i]

    else:
        #Define the wavelength solution onto which we want to interpolate the error bars
        rassine_wav_sol = np.append(rassine_total_lamda[0], rassine_total_lamda[0][-1] + np.diff(rassine_total_lamda[0])[-1])

        #Initialize the Rassine error bars
        rassine_total_norm_err = np.zeros(rassine_total_norm_spctr.shape)

        for i in range(rassine_total_norm_spctr.shape[0]):
            #Define the edges of the wavelength solution.
            base_wav_sol = np.append(tota_lamda[i], tota_lamda[i][-1] + np.diff(tota_lamda[i])[-1])
            rassine_total_norm_err[i] = bind.resampling(rassine_wav_sol, base_wav_sol, tota_err[i], kind='cubic')/rassine_continuum[i]

        
    #Interpolating everything on a common wavelength grid
    #Distinguish two cases depending on the number of modes of observation
    #If there are two modes of observation
    print('INTERPOLATING', '\n')
    if np.sum(tot_mode=='A') != len(tot_mode) and np.sum(tot_mode=='E')!= len(tot_mode):

        min_lamda = max(max(tota_lamda_HA[:, 0]), max(tota_lamda_HE[:, 0]))
        max_lamda = min(min(tota_lamda_HA[:, -1]), min(tota_lamda_HE[:, -1]))

        ref_wave = np.linspace(min_lamda, max_lamda, len(tota_lamda_HA[0]))

        total_lamda_HA = np.zeros(tota_lamda_HA.shape)
        total_lamda_HE = np.zeros(tota_lamda_HE.shape)

        total_norm_spctr_HA = np.zeros(tota_norm_spctr_HA.shape)
        total_norm_spctr_HE = np.zeros(tota_norm_spctr_HE.shape)

        total_norm_err_HA = np.zeros(tota_norm_err_HA.shape)
        total_norm_err_HE = np.zeros(tota_norm_err_HE.shape)

        total_spctr_HA = np.zeros(tota_spctr_HA.shape)
        total_spctr_HE = np.zeros(tota_spctr_HE.shape)

        total_err_HA = np.zeros(tota_err_HA.shape)
        total_err_HE = np.zeros(tota_err_HE.shape)

        #ANTARESS
        #Re-making the wavelength array so that it is the correct format for Antaress
        preliminary_lamda_HA=np.zeros((tota_lamda_HA.shape[0], tota_lamda_HA.shape[1]+1))

        preliminary_lamda_HE=np.zeros((tota_lamda_HE.shape[0], tota_lamda_HE.shape[1]+1))

        for i in range(len(preliminary_lamda_HA)):
            preliminary_lamda_HA[i] = np.append(tota_lamda_HA[i], tota_lamda_HA[i][-1]+np.diff(tota_lamda_HA[i])[-1])

        for i in range(len(preliminary_lamda_HE)):
            preliminary_lamda_HE[i] = np.append(tota_lamda_HE[i], tota_lamda_HE[i][-1]+np.diff(tota_lamda_HE[i])[-1])

        #Re-making the reference wavelength vector for Antaress
        ref_wave_Antaress = np.append(ref_wave, ref_wave[-1]+np.diff(ref_wave)[-1])

        #Making the bins for Rassine (telluric model on rassine wavelength solution)
        #Since all wavelength solutions of Rassine are the same we can set it to any of them
        ref_wave_Rassine_HA = np.append(rassine_total_lamda_HA[0], rassine_total_lamda_HA[0][-1] + np.diff(rassine_total_lamda_HA[0])[-1])

        ref_wave_Rassine_HE = np.append(rassine_total_lamda_HE[0], rassine_total_lamda_HE[0][-1] + np.diff(rassine_total_lamda_HE[0])[-1])

        #Initializing variables
        #Normalized
        total_norm_spctr_Antaress_HA = np.zeros(tota_norm_spctr_HA.shape)
        total_norm_spctr_Antaress_HE = np.zeros(tota_norm_spctr_HE.shape)

        total_norm_err_Antaress_HA = np.zeros(tota_norm_err_HA.shape)
        total_norm_err_Antaress_HE = np.zeros(tota_norm_err_HE.shape)

        #Un-normalized (for SNR calculation)
        total_spctr_Antaress_HA = np.zeros(tota_norm_spctr_HA.shape)
        total_spctr_Antaress_HE = np.zeros(tota_norm_spctr_HE.shape)

        total_err_Antaress_HA = np.zeros(tota_norm_err_HA.shape)
        total_err_Antaress_HE = np.zeros(tota_norm_err_HE.shape)

        total_telluric_HA = np.zeros(tota_telluric_HA.shape)
        total_telluric_HE = np.zeros(tota_telluric_HE.shape)

        rassine_total_telluric_HA = np.zeros(rassine_total_lamda_HA.shape)
        rassine_total_telluric_HE = np.zeros(rassine_total_lamda_HE.shape)

        #HA
        for i in range(len(tota_norm_spctr_HA)):

            interpol_s = interp1d(tota_lamda_HA[i], tota_norm_spctr_HA[i])
            total_norm_spctr_HA[i] = interpol_s(ref_wave)

            interpol_err = interp1d(tota_lamda_HA[i], tota_norm_err_HA[i]**2)
            total_norm_err_HA[i] = np.sqrt(interpol_err(ref_wave))

            interpol_spctr = interp1d(tota_lamda_HA[i], tota_spctr_HA[i])
            total_spctr_HA[i] = interpol_spctr(ref_wave)

            interpol_error = interp1d(tota_lamda_HA[i], tota_err_HA[i]**2)
            total_err_HA[i] = np.sqrt(interpol_error(ref_wave))

            total_lamda_HA[i] = ref_wave

            #Populating ANTARESS arrays
            #Normalized
            testA = bind.resampling(ref_wave_Antaress, preliminary_lamda_HA[i], tota_norm_spctr_HA[i], cov = np.array([tota_norm_err_HA[i]**2]) , kind='cubic')

            total_norm_spctr_Antaress_HA[i] = testA[0]

            total_norm_err_Antaress_HA[i] = np.sqrt(testA[1][0])

            #Un-normalized
            testB = bind.resampling(ref_wave_Antaress, preliminary_lamda_HA[i], tota_spctr_HA[i], cov = np.array([tota_err_HA[i]**2]) , kind='cubic')

            total_spctr_Antaress_HA[i] = testB[0]

            total_err_Antaress_HA[i] = np.sqrt(testB[1][0])

            testC = bind.resampling(ref_wave_Antaress, preliminary_lamda_HA[i], tota_telluric_HA[i], kind='cubic')

            total_telluric_HA[i] = testC

            testZ = bind.resampling(ref_wave_Rassine_HA, preliminary_lamda_HA[i], tota_telluric_HA[i], kind='cubic')

            rassine_total_telluric_HA[i] = testZ

        #HE
        for i in range(len(tota_norm_spctr_HE)):

            interpol_s = interp1d(tota_lamda_HE[i], tota_norm_spctr_HE[i])
            total_norm_spctr_HE[i] = interpol_s(ref_wave)

            interpol_err = interp1d(tota_lamda_HE[i], tota_norm_err_HE[i]**2)
            total_norm_err_HE[i] = np.sqrt(interpol_err(ref_wave))

            interpol_spctr = interp1d(tota_lamda_HE[i], tota_spctr_HE[i])
            total_spctr_HE[i] = interpol_spctr(ref_wave)

            interpol_error = interp1d(tota_lamda_HE[i], tota_err_HE[i]**2)
            total_err_HE[i] = np.sqrt(interpol_error(ref_wave))

            total_lamda_HE[i] = ref_wave

            #Populating ANTARESS arrays
            #Normalized
            testE = bind.resampling(ref_wave_Antaress, preliminary_lamda_HE[i], tota_norm_spctr_HE[i], cov = np.array([tota_norm_err_HE[i]**2]) , kind='cubic')

            total_norm_spctr_Antaress_HE[i] = testE[0]

            total_norm_err_Antaress_HE[i] = np.sqrt(testE[1][0])

            #Un-normalized
            testF = bind.resampling(ref_wave_Antaress, preliminary_lamda_HE[i], tota_spctr_HE[i], cov = np.array([tota_err_HE[i]**2]) , kind='cubic')

            total_spctr_Antaress_HE[i] = testF[0]

            total_err_Antaress_HE[i] = np.sqrt(testF[1][0])

            testD = bind.resampling(ref_wave_Antaress, preliminary_lamda_HE[i], tota_telluric_HE[i] , kind='cubic')

            total_telluric_HE[i] = testD

            testX = bind.resampling(ref_wave_Rassine_HE, preliminary_lamda_HE[i], tota_telluric_HE[i], kind='cubic')

            rassine_total_telluric_HE[i] = testX


    else:

        #Making the bins 
        min_lamda = max(tota_lamda[:, 0])
        max_lamda = min(tota_lamda[:, -1])

        ref_wave = np.linspace(min_lamda, max_lamda, len(tota_lamda[0]))

        #Making the bins for Antaress
        preliminary_lamda=np.zeros((tota_lamda.shape[0], tota_lamda.shape[1]+1))

        for i in range(len(preliminary_lamda)):
            preliminary_lamda[i] = np.append(tota_lamda[i], tota_lamda[i][-1]+np.diff(tota_lamda[i])[-1])

        ref_wave_Antaress = np.append(ref_wave, ref_wave[-1]+np.diff(ref_wave)[-1])

        #Making the bins for Rassine (telluric model on rassine wavelength solution)
        #Since all wavelength solutions of Rassine are the same we can set it to any of them
        ref_wave_Rassine = np.append(rassine_total_lamda[0], rassine_total_lamda[0][-1] + np.diff(rassine_total_lamda[0])[-1])

        #Initializing variables
        total_lamda = np.zeros(tota_lamda.shape)

        total_norm_spctr = np.zeros(tota_norm_spctr.shape)

        total_norm_spctr_Antaress = np.zeros(tota_norm_spctr.shape)

        total_norm_err = np.zeros(tota_norm_err.shape)

        total_norm_err_Antaress = np.zeros(tota_norm_err.shape)

        total_spctr = np.zeros(tota_spctr.shape)

        total_spctr_Antaress = np.zeros(tota_spctr.shape)

        total_err = np.zeros(tota_err.shape)

        total_err_Antaress = np.zeros(tota_err.shape)

        total_telluric = np.zeros(tota_telluric.shape)

        rassine_total_telluric = np.zeros(rassine_total_lamda.shape)

        for i in range(len(tota_lamda)):

            interpol_s = interp1d(tota_lamda[i], tota_norm_spctr[i])
            total_norm_spctr[i] = interpol_s(ref_wave)

            interpol_err = interp1d(tota_lamda[i], tota_norm_err[i]**2)
            total_norm_err[i] = np.sqrt(interpol_err(ref_wave))

            test3 = bind.resampling(ref_wave_Antaress, preliminary_lamda[i], tota_norm_spctr[i], cov = np.array([tota_norm_err[i]**2]) , kind='cubic')

            total_norm_spctr_Antaress[i] = test3[0]

            total_norm_err_Antaress[i] = np.sqrt(test3[1][0])

            test4 = bind.resampling(ref_wave_Antaress, preliminary_lamda[i], tota_spctr[i], cov = np.array([tota_err[i]**2]) , kind='cubic')

            total_spctr_Antaress[i] = test4[0]

            total_err_Antaress[i] = np.sqrt(test4[1][0])

            test5 = bind.resampling(ref_wave_Antaress, preliminary_lamda[i], tota_telluric[i], kind='cubic')

            total_telluric[i] = test5

            test6 = bind.resampling(ref_wave_Rassine, preliminary_lamda[i], tota_telluric[i], kind='cubic')

            rassine_total_telluric[i] = test6

            interpol_spctr = interp1d(tota_lamda[i], tota_spctr[i])
            total_spctr[i] = interpol_spctr(ref_wave)

            interpol_error = interp1d(tota_lamda[i], tota_err[i]**2)
            total_err[i] = np.sqrt(interpol_error(ref_wave))

            total_lamda[i] = ref_wave
            
            
    #Store the relevant stuff in dictionaries
    print('STORAGE', '\n')
    if np.sum(tot_mode=='A') != len(tot_mode) and np.sum(tot_mode=='E')!= len(tot_mode):
        #HA
        rassine_norm_spctr_dict[f"{day} HA"] = rassine_total_norm_spctr_HA
        rassine_norm_err_dict[f"{day} HA"] = rassine_total_norm_err_HA
        rassine_wav_dict[f"{day} HA"] = rassine_total_lamda_HA
        date_dict[f"{day} HA"] = date_HA

        #HE
        rassine_norm_spctr_dict[f"{day} HE"] = rassine_total_norm_spctr_HE
        rassine_norm_err_dict[f"{day} HE"] = rassine_total_norm_err_HE
        rassine_wav_dict[f"{day} HE"] = rassine_total_lamda_HE
        date_dict[f"{day} HE"] = date_HE

    else:
        rassine_norm_spctr_dict[f"{day}"] = rassine_total_norm_spctr
        rassine_norm_err_dict[f"{day}"] = rassine_total_norm_err
        rassine_wav_dict[f"{day}"] = rassine_total_lamda
        date_dict[f"{day}"] = date
        
    mode_dict[f"{day}"] = mode

# Fitting each day

## 2023-01-20

In [ ]:
day = '2023-01-20'

reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

mode = mode_dict[day]

lower_lim = 10827
upper_lim = 10834.1

lower_lim_ew = [10831.5, 10827]
upper_lim_ew = [10834.1, 10831.5]

ref_Si = AF.air2vac(10827.091)

fit_poly_order = 0

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    #HA
    rassine_total_norm_spctr_HA = rassine_norm_spctr_dict[day+' HA']
    rassine_total_norm_err_HA = rassine_norm_err_dict[day+' HA']
    rassine_total_lamda_HA = rassine_wav_dict[day+' HA']
    
    #HE
    rassine_total_norm_spctr_HE = rassine_norm_spctr_dict[day+' HE']
    rassine_total_norm_err_HE = rassine_norm_err_dict[day+' HE']
    rassine_total_lamda_HE = rassine_wav_dict[day+' HE']

    def planetary_model_HA(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HA)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        full_model = model_wings + model_core
        
        return full_model
    
    def planetary_model_HE(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HE)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        full_model = model_wings + model_core
        
        return full_model
    
    
    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']
    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.442, 4.81, 15.03, 16.058, 5350, -1400, -670, 0.9795]
        bounds_param_better = ((4.3, 4.7, 14.9, 16, 5250, -1900, -800, 0.97), (4.7, 4.9, 15.15, 16.4, 6000, 0, -400, 1))

    
    rassine_theta_HA, rassine_err_theta_HA = AF.fit_spctr_line(planetary_model_HA, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                         rassine_total_norm_err_HA, param_name, '', R_power_HA, fit_poly_order)

    
    fit_params_dict[f"{day} HA"] = rassine_theta_HA
    fit_err_dict[f"{day} HA"] = rassine_err_theta_HA

    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.44, 4.8125, 15.04, 16.06, 5200, -1100, -670, 0.9805]
        bounds_param_better = ((4.3, 4.8, 14.9, 16, 5000, -1600, -800, 0.979), (4.7, 4.9, 15.15, 16.4, 6000, 0, -400, 1))

    rassine_theta_HE, rassine_err_theta_HE = AF.fit_spctr_line(planetary_model_HE, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                         rassine_total_norm_err_HE, param_name, '', R_power_HE, fit_poly_order)

    fit_params_dict[f"{day} HE"] = rassine_theta_HE
    fit_err_dict[f"{day} HE"] = rassine_err_theta_HE
 

 #If there is one mode of observation

else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000
        
    rassine_total_norm_spctr = rassine_norm_spctr_dict[day]
    rassine_total_norm_err = rassine_norm_err_dict[day]
    rassine_total_lamda = rassine_wav_dict[day]

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        full_model = model_wings + model_core
        
        return full_model


    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']

        
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.47, 4.84, 14.99, 16.067, 5475, -800, -640, 0.984]
        bounds_param_better = ((4.3, 4.833, 14.9, 16, 5200, -1500, -800, 0.95), (4.7, 4.9, 15.1, 16.4, 6000, 0, -400, 1))

    rassine_theta, rassine_err_theta = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda, rassine_total_norm_spctr, 
                                         rassine_total_norm_err, param_name, '', R_power, fit_poly_order)

    fit_params_dict[f"{day}"] = rassine_theta
    fit_err_dict[f"{day}"] = rassine_err_theta


## 2023-01-21

In [ ]:
day = '2023-01-21'

reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

mode = mode_dict[day]

lower_lim = 10827
upper_lim = 10834.1

lower_lim_ew = [10831.5, 10827]
upper_lim_ew = [10834.1, 10831.5]

ref_Si = AF.air2vac(10827.091)

fit_poly_order = 0

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    #HA
    rassine_total_norm_spctr_HA = rassine_norm_spctr_dict[day+' HA']
    rassine_total_norm_err_HA = rassine_norm_err_dict[day+' HA']
    rassine_total_lamda_HA = rassine_wav_dict[day+' HA']
    
    #HE
    rassine_total_norm_spctr_HE = rassine_norm_spctr_dict[day+' HE']
    rassine_total_norm_err_HE = rassine_norm_err_dict[day+' HE']
    rassine_total_lamda_HE = rassine_wav_dict[day+' HE']

    def planetary_model_HA(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HA)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        full_model = model_wings + model_core
        
        return full_model
    
    def planetary_model_HE(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HE)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        full_model = model_wings + model_core
        
        return full_model
    
    
    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']
    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.44, 4.82, 14.99, 16.06, 5350, -1100, -690, 0.981]
        bounds_param_better = ((4.3, 4.7, 14.9, 16, 5100, -1900, -800, 0.97), (4.7, 4.9, 15.1, 16.1, 6000, 0, -400, 1))

    
    rassine_theta_HA, rassine_err_theta_HA = AF.fit_spctr_line(planetary_model_HA, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                         rassine_total_norm_err_HA, param_name, '', R_power_HA, fit_poly_order)

    
    fit_params_dict[f"{day} HA"] = rassine_theta_HA
    fit_err_dict[f"{day} HA"] = rassine_err_theta_HA

    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.5, 4.8375, 15.05, 16.067, 5550, -1200, -640, 0.983]
        bounds_param_better = ((4.3, 4.8, 14.9, 16, 5000, -1600, -800, 0.979), (4.7, 4.9, 15.15, 16.4, 6000, 0, -400, 1))

    rassine_theta_HE, rassine_err_theta_HE = AF.fit_spctr_line(planetary_model_HE, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                         rassine_total_norm_err_HE, param_name, '', R_power_HE, fit_poly_order)

    fit_params_dict[f"{day} HE"] = rassine_theta_HE
    fit_err_dict[f"{day} HE"] = rassine_err_theta_HE
 

 #If there is one mode of observation

else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000
        
    rassine_total_norm_spctr = rassine_norm_spctr_dict[day]
    rassine_total_norm_err = rassine_norm_err_dict[day]
    rassine_total_lamda = rassine_wav_dict[day]

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        full_model = model_wings + model_core
        
        return full_model


    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']

        
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.47, 4.84, 14.99, 16.067, 5475, -800, -640, 0.984]
        bounds_param_better = ((4.3, 4.833, 14.9, 16, 5200, -1500, -800, 0.95), (4.7, 4.9, 15.1, 16.4, 6000, 0, -400, 1))

    rassine_theta, rassine_err_theta = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda, rassine_total_norm_spctr, 
                                         rassine_total_norm_err, param_name, '', R_power, fit_poly_order)

    fit_params_dict[f"{day}"] = rassine_theta
    fit_err_dict[f"{day}"] = rassine_err_theta


## 2023-01-22

In [ ]:
day = '2023-01-22'

reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

mode = mode_dict[day]

lower_lim = 10827
upper_lim = 10834.1

lower_lim_ew = [10831.5, 10827]
upper_lim_ew = [10834.1, 10831.5]

ref_Si = AF.air2vac(10827.091)

fit_poly_order = 0

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    #HA
    rassine_total_norm_spctr_HA = rassine_norm_spctr_dict[day+' HA']
    rassine_total_norm_err_HA = rassine_norm_err_dict[day+' HA']
    rassine_total_lamda_HA = rassine_wav_dict[day+' HA']
    
    #HE
    rassine_total_norm_spctr_HE = rassine_norm_spctr_dict[day+' HE']
    rassine_total_norm_err_HE = rassine_norm_err_dict[day+' HE']
    rassine_total_lamda_HE = rassine_wav_dict[day+' HE']

    def planetary_model_HA(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HA)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        full_model = model_wings + model_core
        
        return full_model
    
    def planetary_model_HE(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HE)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        full_model = model_wings + model_core
        
        return full_model
    
    
    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']
    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.44, 4.82, 14.99, 16.06, 5350, -1100, -690, 0.981]
        bounds_param_better = ((4.3, 4.7, 14.9, 16, 5100, -1900, -800, 0.97), (4.7, 4.9, 15.1, 16.1, 6000, 0, -400, 1))

    
    rassine_theta_HA, rassine_err_theta_HA = AF.fit_spctr_line(planetary_model_HA, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                         rassine_total_norm_err_HA, param_name, '', R_power_HA, fit_poly_order)

    
    fit_params_dict[f"{day} HA"] = rassine_theta_HA
    fit_err_dict[f"{day} HA"] = rassine_err_theta_HA

    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.5, 4.8375, 15.05, 16.067, 5550, -1200, -640, 0.983]
        bounds_param_better = ((4.3, 4.8, 14.9, 16, 5000, -1600, -800, 0.979), (4.7, 4.9, 15.15, 16.4, 6000, 0, -400, 1))

    rassine_theta_HE, rassine_err_theta_HE = AF.fit_spctr_line(planetary_model_HE, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                         rassine_total_norm_err_HE, param_name, '', R_power_HE, fit_poly_order)

    fit_params_dict[f"{day} HE"] = rassine_theta_HE
    fit_err_dict[f"{day} HE"] = rassine_err_theta_HE
 

 #If there is one mode of observation

else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000
        
    rassine_total_norm_spctr = rassine_norm_spctr_dict[day]
    rassine_total_norm_err = rassine_norm_err_dict[day]
    rassine_total_lamda = rassine_wav_dict[day]

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        full_model = model_wings + model_core
        
        return full_model


    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']

        
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.4, 4.815, 15, 16.0575, 5400, -1400, -700, 0.978]
        bounds_param_better = ((4.25, 4.75, 14.9, 16, 5150, -2500, -800, 0.95), (4.7, 4.87, 15.1, 16.2, 6000, 0, -400, 1))

    rassine_theta, rassine_err_theta = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda, rassine_total_norm_spctr, 
                                         rassine_total_norm_err, param_name, '', R_power, fit_poly_order)

    fit_params_dict[f"{day}"] = rassine_theta
    fit_err_dict[f"{day}"] = rassine_err_theta

## 2023-01-23

In [ ]:
day = '2023-01-23'

reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

mode = mode_dict[day]

lower_lim = 10827
upper_lim = 10834.1

lower_lim_ew = [10831.5, 10827]
upper_lim_ew = [10834.1, 10831.5]

ref_Si = AF.air2vac(10827.091)

fit_poly_order = 0

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    #HA
    rassine_total_norm_spctr_HA = rassine_norm_spctr_dict[day+' HA']
    rassine_total_norm_err_HA = rassine_norm_err_dict[day+' HA']
    rassine_total_lamda_HA = rassine_wav_dict[day+' HA']
    
    #HE
    rassine_total_norm_spctr_HE = rassine_norm_spctr_dict[day+' HE']
    rassine_total_norm_err_HE = rassine_norm_err_dict[day+' HE']
    rassine_total_lamda_HE = rassine_wav_dict[day+' HE']

    def planetary_model_HA(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HA)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        full_model = model_wings + model_core
        
        return full_model
    
    def planetary_model_HE(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HE)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        full_model = model_wings + model_core
        
        return full_model
    
    
    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']
    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.41, 4.821, 15, 16.0635, 5330, -1250, -675, 0.9815]
        bounds_param_better = ((4.3, 4.7, 14.94, 16, 5100, -1800, -800, 0.97), (4.7, 4.9, 15.1, 16.1, 5700, -300, -500, 1))

    
    rassine_theta_HA, rassine_err_theta_HA = AF.fit_spctr_line(planetary_model_HA, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                         rassine_total_norm_err_HA, param_name, '', R_power_HA, fit_poly_order)

    
    fit_params_dict[f"{day} HA"] = rassine_theta_HA
    fit_err_dict[f"{day} HA"] = rassine_err_theta_HA

    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.42, 4.835, 15, 16.066, 5550, -1050, -640, 0.983]
        bounds_param_better = ((4.3, 4.8, 14.9, 16, 5000, -1500, -700, 0.982), (4.7, 4.845, 15.03, 16.069, 6000, -500, -400, 1))

    rassine_theta_HE, rassine_err_theta_HE = AF.fit_spctr_line(planetary_model_HE, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                         rassine_total_norm_err_HE, param_name, '', R_power_HE, fit_poly_order)

    fit_params_dict[f"{day} HE"] = rassine_theta_HE
    fit_err_dict[f"{day} HE"] = rassine_err_theta_HE
 

 #If there is one mode of observation

else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000
        
    rassine_total_norm_spctr = rassine_norm_spctr_dict[day]
    rassine_total_norm_err = rassine_norm_err_dict[day]
    rassine_total_lamda = rassine_wav_dict[day]

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        full_model = model_wings + model_core
        
        return full_model


    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']

        
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.4, 4.815, 15, 16.0575, 5400, -1400, -700, 0.978]
        bounds_param_better = ((4.25, 4.75, 14.9, 16, 5150, -2500, -800, 0.95), (4.7, 4.87, 15.1, 16.2, 6000, 0, -400, 1))

    rassine_theta, rassine_err_theta = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda, rassine_total_norm_spctr, 
                                         rassine_total_norm_err, param_name, '', R_power, fit_poly_order)

    fit_params_dict[f"{day}"] = rassine_theta
    fit_err_dict[f"{day}"] = rassine_err_theta

## 2023-01-24

In [ ]:
day = '2023-01-24'

reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

mode = mode_dict[day]

lower_lim = 10827
upper_lim = 10834.1

lower_lim_ew = [10831.5, 10827]
upper_lim_ew = [10834.1, 10831.5]

ref_Si = AF.air2vac(10827.091)

fit_poly_order = 0

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    #HA
    rassine_total_norm_spctr_HA = rassine_norm_spctr_dict[day+' HA']
    rassine_total_norm_err_HA = rassine_norm_err_dict[day+' HA']
    rassine_total_lamda_HA = rassine_wav_dict[day+' HA']
    
    #HE
    rassine_total_norm_spctr_HE = rassine_norm_spctr_dict[day+' HE']
    rassine_total_norm_err_HE = rassine_norm_err_dict[day+' HE']
    rassine_total_lamda_HE = rassine_wav_dict[day+' HE']

    def planetary_model_HA(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HA)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        full_model = model_wings + model_core
        
        return full_model
    
    def planetary_model_HE(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HE)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        full_model = model_wings + model_core
        
        return full_model
    
    
    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']
    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.41, 4.821, 15, 16.0635, 5330, -1250, -675, 0.9815]
        bounds_param_better = ((4.3, 4.7, 14.94, 16, 5100, -1800, -800, 0.97), (4.7, 4.9, 15.1, 16.1, 5700, -300, -500, 1))

    
    rassine_theta_HA, rassine_err_theta_HA = AF.fit_spctr_line(planetary_model_HA, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                         rassine_total_norm_err_HA, param_name, '', R_power_HA, fit_poly_order)

    
    fit_params_dict[f"{day} HA"] = rassine_theta_HA
    fit_err_dict[f"{day} HA"] = rassine_err_theta_HA

    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.42, 4.835, 15, 16.066, 5550, -1050, -640, 0.983]
        bounds_param_better = ((4.3, 4.8, 14.9, 16, 5000, -1500, -700, 0.982), (4.7, 4.845, 15.03, 16.069, 6000, -500, -400, 1))

    rassine_theta_HE, rassine_err_theta_HE = AF.fit_spctr_line(planetary_model_HE, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                         rassine_total_norm_err_HE, param_name, '', R_power_HE, fit_poly_order)

    fit_params_dict[f"{day} HE"] = rassine_theta_HE
    fit_err_dict[f"{day} HE"] = rassine_err_theta_HE
 

 #If there is one mode of observation

else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000
        
    rassine_total_norm_spctr = rassine_norm_spctr_dict[day]
    rassine_total_norm_err = rassine_norm_err_dict[day]
    rassine_total_lamda = rassine_wav_dict[day]

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        full_model = model_wings + model_core
        
        return full_model


    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']

        
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.38, 4.814, 14.96, 16.061, 5300, -1000, -670, 0.9813]
        bounds_param_better = ((4.3, 4.75, 14.9, 16.05, 5150, -1500, -750, 0.978), (4.5, 4.87, 15.1, 16.1, 6000, -400, -400, 1))

    rassine_theta, rassine_err_theta = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda, rassine_total_norm_spctr, 
                                         rassine_total_norm_err, param_name, '', R_power, fit_poly_order)

    fit_params_dict[f"{day}"] = rassine_theta
    fit_err_dict[f"{day}"] = rassine_err_theta

## 2023-01-25

In [ ]:
day = '2023-01-25'

reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

mode = mode_dict[day]

lower_lim = 10827
upper_lim = 10834.1

lower_lim_ew = [10831.5, 10827]
upper_lim_ew = [10834.1, 10831.5]

ref_Si = AF.air2vac(10827.091)

fit_poly_order = 0

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    #HA
    rassine_total_norm_spctr_HA = rassine_norm_spctr_dict[day+' HA']
    rassine_total_norm_err_HA = rassine_norm_err_dict[day+' HA']
    rassine_total_lamda_HA = rassine_wav_dict[day+' HA']
    
    #HE
    rassine_total_norm_spctr_HE = rassine_norm_spctr_dict[day+' HE']
    rassine_total_norm_err_HE = rassine_norm_err_dict[day+' HE']
    rassine_total_lamda_HE = rassine_wav_dict[day+' HE']

    def planetary_model_HA(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HA)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        full_model = model_wings + model_core
        
        return full_model
    
    def planetary_model_HE(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HE)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        full_model = model_wings + model_core
        
        return full_model
    
    
    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']
    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.41, 4.821, 15, 16.0635, 5330, -1250, -675, 0.9815]
        bounds_param_better = ((4.3, 4.7, 14.94, 16, 5100, -1800, -800, 0.97), (4.7, 4.9, 15.1, 16.1, 5700, -300, -500, 1))

    
    rassine_theta_HA, rassine_err_theta_HA = AF.fit_spctr_line(planetary_model_HA, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                         rassine_total_norm_err_HA, param_name, '', R_power_HA, fit_poly_order)

    
    fit_params_dict[f"{day} HA"] = rassine_theta_HA
    fit_err_dict[f"{day} HA"] = rassine_err_theta_HA

    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.42, 4.835, 15, 16.066, 5550, -1050, -640, 0.983]
        bounds_param_better = ((4.3, 4.8, 14.9, 16, 5000, -1500, -700, 0.982), (4.7, 4.845, 15.03, 16.069, 6000, -500, -400, 1))

    rassine_theta_HE, rassine_err_theta_HE = AF.fit_spctr_line(planetary_model_HE, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                         rassine_total_norm_err_HE, param_name, '', R_power_HE, fit_poly_order)

    fit_params_dict[f"{day} HE"] = rassine_theta_HE
    fit_err_dict[f"{day} HE"] = rassine_err_theta_HE
 

 #If there is one mode of observation

else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000
        
    rassine_total_norm_spctr = rassine_norm_spctr_dict[day]
    rassine_total_norm_err = rassine_norm_err_dict[day]
    rassine_total_lamda = rassine_wav_dict[day]

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        full_model = model_wings + model_core
        
        return full_model


    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']

        
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.54, 4.819, 14.98, 16.063, 5350, -1000, -670, 0.98]
        bounds_param_better = ((4.3, 4.75, 14.9, 16.05, 5150, -1800, -750, 0.975), (4.7, 4.87, 15.1, 16.1, 5700, 0, -400, 1))

    rassine_theta, rassine_err_theta = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda, rassine_total_norm_spctr, 
                                         rassine_total_norm_err, param_name, '', R_power, fit_poly_order)

    fit_params_dict[f"{day}"] = rassine_theta
    fit_err_dict[f"{day}"] = rassine_err_theta

## 2023-01-26

In [ ]:
day = '2023-01-26'

reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

mode = mode_dict[day]

lower_lim = 10827
upper_lim = 10834.1

lower_lim_ew = [10831.5, 10827]
upper_lim_ew = [10834.1, 10831.5]

ref_Si = AF.air2vac(10827.091)

fit_poly_order = 0

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    #HA
    rassine_total_norm_spctr_HA = rassine_norm_spctr_dict[day+' HA']
    rassine_total_norm_err_HA = rassine_norm_err_dict[day+' HA']
    rassine_total_lamda_HA = rassine_wav_dict[day+' HA']
    
    #HE
    rassine_total_norm_spctr_HE = rassine_norm_spctr_dict[day+' HE']
    rassine_total_norm_err_HE = rassine_norm_err_dict[day+' HE']
    rassine_total_lamda_HE = rassine_wav_dict[day+' HE']

    def planetary_model_HA(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HA)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        full_model = model_wings + model_core
        
        return full_model
    
    def planetary_model_HE(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HE)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        full_model = model_wings + model_core
        
        return full_model
    
    
    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']
    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.41, 4.821, 15, 16.0635, 5330, -1250, -675, 0.9815]
        bounds_param_better = ((4.3, 4.7, 14.94, 16, 5100, -1800, -800, 0.97), (4.7, 4.9, 15.1, 16.1, 5700, -300, -500, 1))

    
    rassine_theta_HA, rassine_err_theta_HA = AF.fit_spctr_line(planetary_model_HA, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                         rassine_total_norm_err_HA, param_name, '', R_power_HA, fit_poly_order)

    
    fit_params_dict[f"{day} HA"] = rassine_theta_HA
    fit_err_dict[f"{day} HA"] = rassine_err_theta_HA

    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.42, 4.835, 15, 16.066, 5550, -1050, -640, 0.983]
        bounds_param_better = ((4.3, 4.8, 14.9, 16, 5000, -1500, -700, 0.982), (4.7, 4.845, 15.03, 16.069, 6000, -500, -400, 1))

    rassine_theta_HE, rassine_err_theta_HE = AF.fit_spctr_line(planetary_model_HE, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                         rassine_total_norm_err_HE, param_name, '', R_power_HE, fit_poly_order)

    fit_params_dict[f"{day} HE"] = rassine_theta_HE
    fit_err_dict[f"{day} HE"] = rassine_err_theta_HE
 

 #If there is one mode of observation

else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000
        
    rassine_total_norm_spctr = rassine_norm_spctr_dict[day]
    rassine_total_norm_err = rassine_norm_err_dict[day]
    rassine_total_lamda = rassine_wav_dict[day]

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        full_model = model_wings + model_core
        
        return full_model


    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']

        
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.46, 4.845, 14.99, 16.068, 5475, -800, -640, 0.984]
        bounds_param_better = ((4.3, 4.8, 14.9, 16.06, 5300, -1600, -700, 0.98), (4.7, 4.89, 15.1, 16.1, 5800, 0, -580, 1))

    rassine_theta, rassine_err_theta = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda, rassine_total_norm_spctr, 
                                         rassine_total_norm_err, param_name, '', R_power, fit_poly_order)

    fit_params_dict[f"{day}"] = rassine_theta
    fit_err_dict[f"{day}"] = rassine_err_theta

## 2023-01-27

In [ ]:
day = '2023-01-27'

reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

mode = mode_dict[day]

lower_lim = 10827
upper_lim = 10834.1

lower_lim_ew = [10831.5, 10827]
upper_lim_ew = [10834.1, 10831.5]

ref_Si = AF.air2vac(10827.091)

fit_poly_order = 0

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    #HA
    rassine_total_norm_spctr_HA = rassine_norm_spctr_dict[day+' HA']
    rassine_total_norm_err_HA = rassine_norm_err_dict[day+' HA']
    rassine_total_lamda_HA = rassine_wav_dict[day+' HA']
    
    #HE
    rassine_total_norm_spctr_HE = rassine_norm_spctr_dict[day+' HE']
    rassine_total_norm_err_HE = rassine_norm_err_dict[day+' HE']
    rassine_total_lamda_HE = rassine_wav_dict[day+' HE']

    def planetary_model_HA(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HA)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        full_model = model_wings + model_core
        
        return full_model
    
    def planetary_model_HE(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HE)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        full_model = model_wings + model_core
        
        return full_model
    
    
    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']
    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.41, 4.821, 15, 16.0635, 5330, -1250, -675, 0.9815]
        bounds_param_better = ((4.3, 4.7, 14.94, 16, 5100, -1800, -800, 0.97), (4.7, 4.9, 15.1, 16.1, 5700, -300, -500, 1))

    
    rassine_theta_HA, rassine_err_theta_HA = AF.fit_spctr_line(planetary_model_HA, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                         rassine_total_norm_err_HA, param_name, '', R_power_HA, fit_poly_order)

    
    fit_params_dict[f"{day} HA"] = rassine_theta_HA
    fit_err_dict[f"{day} HA"] = rassine_err_theta_HA

    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.42, 4.835, 15, 16.066, 5550, -1050, -640, 0.983]
        bounds_param_better = ((4.3, 4.8, 14.9, 16, 5000, -1500, -700, 0.982), (4.7, 4.845, 15.03, 16.069, 6000, -500, -400, 1))

    rassine_theta_HE, rassine_err_theta_HE = AF.fit_spctr_line(planetary_model_HE, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                         rassine_total_norm_err_HE, param_name, '', R_power_HE, fit_poly_order)

    fit_params_dict[f"{day} HE"] = rassine_theta_HE
    fit_err_dict[f"{day} HE"] = rassine_err_theta_HE
 

 #If there is one mode of observation

else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000
        
    rassine_total_norm_spctr = rassine_norm_spctr_dict[day]
    rassine_total_norm_err = rassine_norm_err_dict[day]
    rassine_total_lamda = rassine_wav_dict[day]

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        full_model = model_wings + model_core
        
        return full_model


    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']

        
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.45, 4.8425, 14.98, 16.0685, 5625, -1100, -650, 0.9847]
        bounds_param_better = ((4.3, 4.83, 14.92, 16.06, 5500, -1500, -700, 0.98), (4.7, 4.86, 15.05, 16.1, 5850, -400, -580, 1))

    rassine_theta, rassine_err_theta = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda, rassine_total_norm_spctr, 
                                         rassine_total_norm_err, param_name, '', R_power, fit_poly_order)

    fit_params_dict[f"{day}"] = rassine_theta
    fit_err_dict[f"{day}"] = rassine_err_theta

## 2023-02-01

In [ ]:
day = '2023-02-01'

reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

mode = mode_dict[day]

lower_lim = 10827
upper_lim = 10834.1

lower_lim_ew = [10831.5, 10827]
upper_lim_ew = [10834.1, 10831.5]

ref_Si = AF.air2vac(10827.091)

fit_poly_order = 0

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    #HA
    rassine_total_norm_spctr_HA = rassine_norm_spctr_dict[day+' HA']
    rassine_total_norm_err_HA = rassine_norm_err_dict[day+' HA']
    rassine_total_lamda_HA = rassine_wav_dict[day+' HA']
    
    #HE
    rassine_total_norm_spctr_HE = rassine_norm_spctr_dict[day+' HE']
    rassine_total_norm_err_HE = rassine_norm_err_dict[day+' HE']
    rassine_total_lamda_HE = rassine_wav_dict[day+' HE']

    def planetary_model_HA(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HA)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        full_model = model_wings + model_core
        
        return full_model
    
    def planetary_model_HE(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HE)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        full_model = model_wings + model_core
        
        return full_model
    
    
    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']
    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.41, 4.821, 15, 16.0635, 5330, -1250, -675, 0.9815]
        bounds_param_better = ((4.3, 4.7, 14.94, 16, 5100, -1800, -800, 0.97), (4.7, 4.9, 15.1, 16.1, 5700, -300, -500, 1))

    
    rassine_theta_HA, rassine_err_theta_HA = AF.fit_spctr_line(planetary_model_HA, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                         rassine_total_norm_err_HA, param_name, '', R_power_HA, fit_poly_order)

    
    fit_params_dict[f"{day} HA"] = rassine_theta_HA
    fit_err_dict[f"{day} HA"] = rassine_err_theta_HA

    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.42, 4.835, 15, 16.066, 5550, -1050, -640, 0.983]
        bounds_param_better = ((4.3, 4.8, 14.9, 16, 5000, -1500, -700, 0.982), (4.7, 4.845, 15.03, 16.069, 6000, -500, -400, 1))

    rassine_theta_HE, rassine_err_theta_HE = AF.fit_spctr_line(planetary_model_HE, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                         rassine_total_norm_err_HE, param_name, '', R_power_HE, fit_poly_order)

    fit_params_dict[f"{day} HE"] = rassine_theta_HE
    fit_err_dict[f"{day} HE"] = rassine_err_theta_HE
 

 #If there is one mode of observation

else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000
        
    rassine_total_norm_spctr = rassine_norm_spctr_dict[day]
    rassine_total_norm_err = rassine_norm_err_dict[day]
    rassine_total_lamda = rassine_wav_dict[day]

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        full_model = model_wings + model_core
        
        return full_model


    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']

        
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.45, 4.7, 15, 16.0665, 5300, -1100, -680, 0.9833]
        bounds_param_better = ((4.3, 4.6, 14.8, 16, 5100, -1500, -800, 0.95), (4.7, 4.9, 15.1, 16.4, 6000, -200, -400, 1))

    rassine_theta, rassine_err_theta = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda, rassine_total_norm_spctr, 
                                         rassine_total_norm_err, param_name, '', R_power, fit_poly_order)

    fit_params_dict[f"{day}"] = rassine_theta
    fit_err_dict[f"{day}"] = rassine_err_theta

## 2023-02-02

In [ ]:
day = '2023-02-02'

reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

mode = mode_dict[day]

lower_lim = 10827
upper_lim = 10834.1

lower_lim_ew = [10831.5, 10827]
upper_lim_ew = [10834.1, 10831.5]

ref_Si = AF.air2vac(10827.091)

fit_poly_order = 0

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    #HA
    rassine_total_norm_spctr_HA = rassine_norm_spctr_dict[day+' HA']
    rassine_total_norm_err_HA = rassine_norm_err_dict[day+' HA']
    rassine_total_lamda_HA = rassine_wav_dict[day+' HA']
    
    #HE
    rassine_total_norm_spctr_HE = rassine_norm_spctr_dict[day+' HE']
    rassine_total_norm_err_HE = rassine_norm_err_dict[day+' HE']
    rassine_total_lamda_HE = rassine_wav_dict[day+' HE']

    def planetary_model_HA(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HA)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        full_model = model_wings + model_core
        
        return full_model
    
    def planetary_model_HE(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HE)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        full_model = model_wings + model_core
        
        return full_model
    
    
    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']
    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.41, 4.821, 15, 16.0635, 5330, -1250, -675, 0.9815]
        bounds_param_better = ((4.3, 4.7, 14.94, 16, 5100, -1800, -800, 0.97), (4.7, 4.9, 15.1, 16.1, 5700, -300, -500, 1))

    
    rassine_theta_HA, rassine_err_theta_HA = AF.fit_spctr_line(planetary_model_HA, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                         rassine_total_norm_err_HA, param_name, '', R_power_HA, fit_poly_order)

    
    fit_params_dict[f"{day} HA"] = rassine_theta_HA
    fit_err_dict[f"{day} HA"] = rassine_err_theta_HA

    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.42, 4.835, 15, 16.066, 5550, -1050, -640, 0.983]
        bounds_param_better = ((4.3, 4.8, 14.9, 16, 5000, -1500, -700, 0.982), (4.7, 4.845, 15.03, 16.069, 6000, -500, -400, 1))

    rassine_theta_HE, rassine_err_theta_HE = AF.fit_spctr_line(planetary_model_HE, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                         rassine_total_norm_err_HE, param_name, '', R_power_HE, fit_poly_order)

    fit_params_dict[f"{day} HE"] = rassine_theta_HE
    fit_err_dict[f"{day} HE"] = rassine_err_theta_HE
 

 #If there is one mode of observation

else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000
        
    rassine_total_norm_spctr = rassine_norm_spctr_dict[day]
    rassine_total_norm_err = rassine_norm_err_dict[day]
    rassine_total_lamda = rassine_wav_dict[day]

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        full_model = model_wings + model_core
        
        return full_model


    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']

        
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.43, 4.8, 15, 16.064, 5250, -900, -670, 0.982]
        bounds_param_better = ((4.3, 4.7, 14.8, 16.061, 5100, -1500, -800, 0.96), (4.6, 4.83, 15.1, 16.068, 5350, -200, -400, 1))

    rassine_theta, rassine_err_theta = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda, rassine_total_norm_spctr, 
                                         rassine_total_norm_err, param_name, '', R_power, fit_poly_order)

    fit_params_dict[f"{day}"] = rassine_theta
    fit_err_dict[f"{day}"] = rassine_err_theta

## 2023-02-03

In [ ]:
day = '2023-02-03'

reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

mode = mode_dict[day]

lower_lim = 10827
upper_lim = 10834.1

lower_lim_ew = [10831.5, 10827]
upper_lim_ew = [10834.1, 10831.5]

ref_Si = AF.air2vac(10827.091)

fit_poly_order = 0

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    #HA
    rassine_total_norm_spctr_HA = rassine_norm_spctr_dict[day+' HA']
    rassine_total_norm_err_HA = rassine_norm_err_dict[day+' HA']
    rassine_total_lamda_HA = rassine_wav_dict[day+' HA']
    
    #HE
    rassine_total_norm_spctr_HE = rassine_norm_spctr_dict[day+' HE']
    rassine_total_norm_err_HE = rassine_norm_err_dict[day+' HE']
    rassine_total_lamda_HE = rassine_wav_dict[day+' HE']

    def planetary_model_HA(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HA)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        full_model = model_wings + model_core
        
        return full_model
    
    def planetary_model_HE(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HE)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        full_model = model_wings + model_core
        
        return full_model
    
    
    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']
    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.41, 4.821, 15, 16.0635, 5330, -1250, -675, 0.9815]
        bounds_param_better = ((4.3, 4.7, 14.94, 16, 5100, -1800, -800, 0.97), (4.7, 4.9, 15.1, 16.1, 5700, -300, -500, 1))

    
    rassine_theta_HA, rassine_err_theta_HA = AF.fit_spctr_line(planetary_model_HA, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                         rassine_total_norm_err_HA, param_name, '', R_power_HA, fit_poly_order)

    
    fit_params_dict[f"{day} HA"] = rassine_theta_HA
    fit_err_dict[f"{day} HA"] = rassine_err_theta_HA

    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.42, 4.835, 15, 16.066, 5550, -1050, -640, 0.983]
        bounds_param_better = ((4.3, 4.8, 14.9, 16, 5000, -1500, -700, 0.982), (4.7, 4.845, 15.03, 16.069, 6000, -500, -400, 1))

    rassine_theta_HE, rassine_err_theta_HE = AF.fit_spctr_line(planetary_model_HE, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                         rassine_total_norm_err_HE, param_name, '', R_power_HE, fit_poly_order)

    fit_params_dict[f"{day} HE"] = rassine_theta_HE
    fit_err_dict[f"{day} HE"] = rassine_err_theta_HE
 

 #If there is one mode of observation

else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000
        
    rassine_total_norm_spctr = rassine_norm_spctr_dict[day]
    rassine_total_norm_err = rassine_norm_err_dict[day]
    rassine_total_lamda = rassine_wav_dict[day]

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        full_model = model_wings + model_core
        
        return full_model


    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']

        
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.44, 4.817, 15.01, 16.064, 5270, -1200, -670, 0.9825]
        bounds_param_better = ((4.3, 4.7, 14.8, 16, 5100, -1700, -800, 0.96), (4.6, 4.84, 15.1, 16.1, 5500, -200, -400, 1))
        
    rassine_theta, rassine_err_theta = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda, rassine_total_norm_spctr, 
                                         rassine_total_norm_err, param_name, '', R_power, fit_poly_order)

    fit_params_dict[f"{day}"] = rassine_theta
    fit_err_dict[f"{day}"] = rassine_err_theta

## 2023-02-06

In [ ]:
day = '2023-02-06'

reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

mode = mode_dict[day]

lower_lim = 10827
upper_lim = 10834.1

lower_lim_ew = [10831.5, 10827]
upper_lim_ew = [10834.1, 10831.5]

ref_Si = AF.air2vac(10827.091)

fit_poly_order = 0

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    #HA
    rassine_total_norm_spctr_HA = rassine_norm_spctr_dict[day+' HA']
    rassine_total_norm_err_HA = rassine_norm_err_dict[day+' HA']
    rassine_total_lamda_HA = rassine_wav_dict[day+' HA']
    
    #HE
    rassine_total_norm_spctr_HE = rassine_norm_spctr_dict[day+' HE']
    rassine_total_norm_err_HE = rassine_norm_err_dict[day+' HE']
    rassine_total_lamda_HE = rassine_wav_dict[day+' HE']

    def planetary_model_HA(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HA)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        full_model = model_wings + model_core
        
        return full_model
    
    def planetary_model_HE(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HE)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        full_model = model_wings + model_core
        
        return full_model
    
    
    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']
    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.41, 4.821, 15, 16.0635, 5330, -1250, -675, 0.9815]
        bounds_param_better = ((4.3, 4.7, 14.94, 16, 5100, -1800, -800, 0.97), (4.7, 4.9, 15.1, 16.1, 5700, -300, -500, 1))

    
    rassine_theta_HA, rassine_err_theta_HA = AF.fit_spctr_line(planetary_model_HA, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                         rassine_total_norm_err_HA, param_name, '', R_power_HA, fit_poly_order)

    
    fit_params_dict[f"{day} HA"] = rassine_theta_HA
    fit_err_dict[f"{day} HA"] = rassine_err_theta_HA

    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.42, 4.835, 15, 16.066, 5550, -1050, -640, 0.983]
        bounds_param_better = ((4.3, 4.8, 14.9, 16, 5000, -1500, -700, 0.982), (4.7, 4.845, 15.03, 16.069, 6000, -500, -400, 1))

    rassine_theta_HE, rassine_err_theta_HE = AF.fit_spctr_line(planetary_model_HE, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                         rassine_total_norm_err_HE, param_name, '', R_power_HE, fit_poly_order)

    fit_params_dict[f"{day} HE"] = rassine_theta_HE
    fit_err_dict[f"{day} HE"] = rassine_err_theta_HE
 

 #If there is one mode of observation

else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000
        
    rassine_total_norm_spctr = rassine_norm_spctr_dict[day]
    rassine_total_norm_err = rassine_norm_err_dict[day]
    rassine_total_lamda = rassine_wav_dict[day]

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        full_model = model_wings + model_core
        
        return full_model


    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']

        
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.43, 4.817, 15.035, 16.062, 5150, -1100, -685, 0.9827]
        bounds_param_better = ((4.3, 4.7, 14.8, 15.9, 5000, -1700, -750, 0.96), (4.6, 4.84, 15.1, 16.065, 5500, -200, -400, 1))
        
    rassine_theta, rassine_err_theta = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda, rassine_total_norm_spctr, 
                                         rassine_total_norm_err, param_name, '', R_power, fit_poly_order)

    fit_params_dict[f"{day}"] = rassine_theta
    fit_err_dict[f"{day}"] = rassine_err_theta

## 2023-02-15

In [ ]:
day = '2023-02-15'

reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

mode = mode_dict[day]

lower_lim = 10827
upper_lim = 10834.1

lower_lim_ew = [10831.5, 10827]
upper_lim_ew = [10834.1, 10831.5]

ref_Si = AF.air2vac(10827.091)

fit_poly_order = 0

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    #HA
    rassine_total_norm_spctr_HA = rassine_norm_spctr_dict[day+' HA']
    rassine_total_norm_err_HA = rassine_norm_err_dict[day+' HA']
    rassine_total_lamda_HA = rassine_wav_dict[day+' HA']
    
    #HE
    rassine_total_norm_spctr_HE = rassine_norm_spctr_dict[day+' HE']
    rassine_total_norm_err_HE = rassine_norm_err_dict[day+' HE']
    rassine_total_lamda_HE = rassine_wav_dict[day+' HE']

    def planetary_model_HA(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HA)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        full_model = model_wings + model_core
        
        return full_model
    
    def planetary_model_HE(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HE)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        full_model = model_wings + model_core
        
        return full_model
    
    
    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']
    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.41, 4.821, 15, 16.0635, 5330, -1250, -675, 0.9815]
        bounds_param_better = ((4.3, 4.7, 14.94, 16, 5100, -1800, -800, 0.97), (4.7, 4.9, 15.1, 16.1, 5700, -300, -500, 1))

    
    rassine_theta_HA, rassine_err_theta_HA = AF.fit_spctr_line(planetary_model_HA, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                         rassine_total_norm_err_HA, param_name, '', R_power_HA, fit_poly_order)

    
    fit_params_dict[f"{day} HA"] = rassine_theta_HA
    fit_err_dict[f"{day} HA"] = rassine_err_theta_HA

    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.42, 4.835, 15, 16.066, 5550, -1050, -640, 0.983]
        bounds_param_better = ((4.3, 4.8, 14.9, 16, 5000, -1500, -700, 0.982), (4.7, 4.845, 15.03, 16.069, 6000, -500, -400, 1))

    rassine_theta_HE, rassine_err_theta_HE = AF.fit_spctr_line(planetary_model_HE, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                         rassine_total_norm_err_HE, param_name, '', R_power_HE, fit_poly_order)

    fit_params_dict[f"{day} HE"] = rassine_theta_HE
    fit_err_dict[f"{day} HE"] = rassine_err_theta_HE
 

 #If there is one mode of observation

else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000
        
    rassine_total_norm_spctr = rassine_norm_spctr_dict[day]
    rassine_total_norm_err = rassine_norm_err_dict[day]
    rassine_total_lamda = rassine_wav_dict[day]

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        full_model = model_wings + model_core
        
        return full_model


    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']

        
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.41, 4.817, 15.03, 16.065, 5230, -1000, -660, 0.9833]
        bounds_param_better = ((4.3, 4.8, 14.9, 16.05, 5100, -1500, -720, 0.98), (4.55, 4.84, 15.1, 16.07, 5500, -400, -550, 1))
        
    rassine_theta, rassine_err_theta = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda, rassine_total_norm_spctr, 
                                         rassine_total_norm_err, param_name, '', R_power, fit_poly_order)

    fit_params_dict[f"{day}"] = rassine_theta
    fit_err_dict[f"{day}"] = rassine_err_theta

## 2023-02-16

In [ ]:
day = '2023-02-16'

reload(AF)
os.chdir('/Users/samsonmercier/Desktop/UNIGE/Winter_Semester_2022-2023/APLII/he_triplet')
reload(spec_mod)

mode = mode_dict[day]

lower_lim = 10827
upper_lim = 10834.1

lower_lim_ew = [10831.5, 10827]
upper_lim_ew = [10834.1, 10831.5]

ref_Si = AF.air2vac(10827.091)

fit_poly_order = 0

if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    R_power_HE = 74000
    R_power_HA = 86000

    #HA
    rassine_total_norm_spctr_HA = rassine_norm_spctr_dict[day+' HA']
    rassine_total_norm_err_HA = rassine_norm_err_dict[day+' HA']
    rassine_total_lamda_HA = rassine_wav_dict[day+' HA']
    
    #HE
    rassine_total_norm_spctr_HE = rassine_norm_spctr_dict[day+' HE']
    rassine_total_norm_err_HE = rassine_norm_err_dict[day+' HE']
    rassine_total_lamda_HE = rassine_wav_dict[day+' HE']

    def planetary_model_HA(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HA)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HA)

        full_model = model_wings + model_core
        
        return full_model
    
    def planetary_model_HE(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power_HE)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power_HE)

        full_model = model_wings + model_core
        
        return full_model
    
    
    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']
    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.41, 4.821, 15, 16.0635, 5330, -1250, -675, 0.9815]
        bounds_param_better = ((4.3, 4.7, 14.94, 16, 5100, -1800, -800, 0.97), (4.7, 4.9, 15.1, 16.1, 5700, -300, -500, 1))

    
    rassine_theta_HA, rassine_err_theta_HA = AF.fit_spctr_line(planetary_model_HA, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HA, rassine_total_norm_spctr_HA, 
                                         rassine_total_norm_err_HA, param_name, '', R_power_HA, fit_poly_order)

    
    fit_params_dict[f"{day} HA"] = rassine_theta_HA
    fit_err_dict[f"{day} HA"] = rassine_err_theta_HA

    
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.42, 4.835, 15, 16.066, 5550, -1050, -640, 0.983]
        bounds_param_better = ((4.3, 4.8, 14.9, 16, 5000, -1500, -700, 0.982), (4.7, 4.845, 15.03, 16.069, 6000, -500, -400, 1))

    rassine_theta_HE, rassine_err_theta_HE = AF.fit_spctr_line(planetary_model_HE, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda_HE, rassine_total_norm_spctr_HE, 
                                         rassine_total_norm_err_HE, param_name, '', R_power_HE, fit_poly_order)

    fit_params_dict[f"{day} HE"] = rassine_theta_HE
    fit_err_dict[f"{day} HE"] = rassine_err_theta_HE
 

 #If there is one mode of observation

else:
    if mode[0] == 'A':
        R_power = 86000
    else:
        R_power = 74000
        
    rassine_total_norm_spctr = rassine_norm_spctr_dict[day]
    rassine_total_norm_err = rassine_norm_err_dict[day]
    rassine_total_lamda = rassine_wav_dict[day]

    def planetary_model(x, log_temp_He, log_temp_Si, log_density_He, log_density_Si, Si_d, RV_offset_He, RV_offset_Si, A):
        
        wav_overlap1, wav_overlap2 = AF.range_calculator(log_temp_Si, log_density_Si, Si_d, RV_offset_Si, R_power)
        #Describing the continuum with a polynomial
        if fit_poly_order == 0:
            poly = A*np.ones(x.shape)
        elif fit_poly_order == 2:
            poly = A + B*x + C*x**2
        elif fit_poly_order == 1:
            poly = A + B*x
        
        #Separating the flux described by a polynomial into wings and cores
        poly_core = np.zeros(x.shape)
        poly_wings = np.zeros(x.shape)
        
        for i, j in enumerate(x):
            if lower_lim <= j <= wav_overlap1 or wav_overlap2 <= j <= upper_lim:
                poly_wings[i] = poly[i]
            elif wav_overlap1 <= j <= wav_overlap2:
                poly_core[i] = poly[i]

        #Putting model together
        model_wings = spec_mod.spectrum_he_and_Si(x, poly_wings,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', Si_d, RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        model_core = spec_mod.spectrum_he_and_Si_Gauss(x, poly_core,  10**log_temp_He, 10**log_temp_Si, 
                           10**log_density_He, 10**log_density_Si, None, 'VACUUM', RV_offset_He, 
                            RV_offset_Si, R_pow = R_power)

        full_model = model_wings + model_core
        
        return full_model


    if fit_poly_order == 2:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B', 'C']
    
    elif fit_poly_order == 1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A', 'B']

    elif fit_poly_order == 0:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    
    elif fit_poly_order == -1:
        param_name = ['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si']

        
    #['log_temp_He', 'log_temp_Si', 'log_density_He', 'log_density_Si', 'Si_d', 'RV_offset_He', 'RV_offset_Si', 'A']
    if fit_poly_order == 0:
        guess_params = [4.47, 4.816, 15.05, 16.064, 5200, -1100, -680, 0.985]
        bounds_param_better = ((4.3, 4.8, 14.9, 16.05, 5000, -1600, -750, 0.98), (4.6, 4.84, 15.12, 16.072, 5500, -200, -500, 1))
        
    rassine_theta, rassine_err_theta = AF.fit_spctr_line(planetary_model, lower_lim, upper_lim, lower_lim_ew, upper_lim_ew, 
                                         guess_params, bounds_param_better, rassine_total_lamda, rassine_total_norm_spctr, 
                                         rassine_total_norm_err, param_name, '', R_power, fit_poly_order)

    fit_params_dict[f"{day}"] = rassine_theta
    fit_err_dict[f"{day}"] = rassine_err_theta

In [ ]:
save=False 
num_bins = 20


if np.sum(mode=='A') != len(mode) and np.sum(mode=='E')!= len(mode):
    date_HA = date_dict[day+' HA']
    date_HE = date_dict[day+' HE']
    
    ref_time = mt.floor(date_HA[0])

    #HA
    theta_in_question = rassine_theta_HA
    err_in_question = rassine_err_theta_HA
    date_in_question = date_HA

    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]

    new_param_name = param_name + ['He EW', 'Si EW']

    fig, axs = plt.subplots (mt.ceil(theta_in_question.shape[1]/2), 2, sharex=True, figsize=[12, 14])
    ax = axs.ravel()

    for i in range(theta_in_question.shape[1]):
        avg = np.average(theta_in_question[:, i])
        std = np.sqrt(np.average((theta_in_question[:, i] - avg)**2))

        binned_data = ss.binned_statistic(date_in_question, theta_in_question[:, i], statistic=np.ma.median, bins=num_bins)[0]

        ax[i].plot(date_in_question - ref_time, theta_in_question[:, i], '.', alpha=0.2)
        ax[i].plot(binned_date - ref_time, binned_data, 'r.', markersize=10)

        textstr = '$\sigma$ = %.4f'%std
        ax[i].text(0.04, 0.90, textstr, transform=ax[i].transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

        ax[i].set_ylabel(new_param_name[i], fontsize=14)
        ax[i].tick_params(axis='y', labelsize=14)
        if i == theta_in_question.shape[1]-1 or i == theta_in_question.shape[1]-2:
            ax[i].tick_params(axis='x', labelsize=14)
            ax[i].set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)

    if not ax[-1].lines: 
        ax[-1].set_visible(False)


    fig.tight_layout()
    plt.subplots_adjust(hspace=0)
    if save:
        plt.savefig(save_location+'Rassine_Timeseries_'+str(fit_poly_order)+'th_order_HA.pdf')
    
    #HE
    theta_in_question = rassine_theta_HE
    err_in_question = rassine_err_theta_HE
    date_in_question = date_HE

    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]

    new_param_name = param_name + ['He EW', 'Si EW']

    fig, axs = plt.subplots (mt.ceil(theta_in_question.shape[1]/2), 2, sharex=True, figsize=[12, 14])
    ax = axs.ravel()

    for i in range(theta_in_question.shape[1]):
        avg = np.average(theta_in_question[:, i])
        std = np.sqrt(np.average((theta_in_question[:, i] - avg)**2))

        binned_data = ss.binned_statistic(date_in_question, theta_in_question[:, i], statistic=np.ma.median, bins=num_bins)[0]

        ax[i].plot(date_in_question - ref_time, theta_in_question[:, i], '.', alpha=0.2)
        ax[i].plot(binned_date - ref_time, binned_data, 'r.', markersize=10)

        textstr = '$\sigma$ = %.4f'%std
        ax[i].text(0.04, 0.90, textstr, transform=ax[i].transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

        ax[i].set_ylabel(new_param_name[i], fontsize=14)
        ax[i].tick_params(axis='y', labelsize=14)
        if i == theta_in_question.shape[1]-1 or i == theta_in_question.shape[1]-2:
            ax[i].tick_params(axis='x', labelsize=14)
            ax[i].set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)

    if not ax[-1].lines: 
        ax[-1].set_visible(False)


    fig.tight_layout()
    plt.subplots_adjust(hspace=0)
    if save:
        plt.savefig(save_location+'Rassine_Timeseries_'+str(fit_poly_order)+'th_order_HE.pdf')
        
else:
    date = date_dict[day]
    
    ref_time = mt.floor(date[0])
    
    theta_in_question = rassine_theta
    err_in_question = rassine_err_theta
    date_in_question = date

    binned_date = ss.binned_statistic(date_in_question, date_in_question, statistic=np.ma.median, bins=num_bins)[0]

    new_param_name = param_name + ['He EW', 'Si EW']

    fig, axs = plt.subplots (mt.ceil(theta_in_question.shape[1]/2), 2, sharex=True, figsize=[12, 14])
    ax = axs.ravel()

    for i in range(theta_in_question.shape[1]):
        avg = np.average(theta_in_question[:, i])
        std = np.sqrt(np.average((theta_in_question[:, i] - avg)**2))

        binned_data = ss.binned_statistic(date_in_question, theta_in_question[:, i], statistic=np.ma.median, bins=num_bins)[0]

        ax[i].plot(date_in_question - ref_time, theta_in_question[:, i], '.', alpha=0.2)
        ax[i].plot(binned_date - ref_time, binned_data, 'r.', markersize=10)

        textstr = '$\sigma$ = %.4f'%std
        ax[i].text(0.04, 0.90, textstr, transform=ax[i].transAxes, fontsize=14, bbox = dict(facecolor='white', alpha=0.5))

        ax[i].set_ylabel(new_param_name[i], fontsize=14)
        ax[i].tick_params(axis='y', labelsize=14)
        if i == theta_in_question.shape[1]-1 or i == theta_in_question.shape[1]-2:
            ax[i].tick_params(axis='x', labelsize=14)
            ax[i].set_xlabel('Time (MJD-'+str(ref_time)+')', fontsize=14)

    if not ax[-1].lines: 
        ax[-1].set_visible(False)


    fig.tight_layout()
    plt.subplots_adjust(hspace=0)
    if save:
        plt.savefig(save_location+'Rassine_Timeseries_'+str(fit_poly_order)+'th_order.pdf')

# Plot all the days

## Main Plot

In [ ]:
reload(AF)

# List of red shades
day_color = [
    "#FF0000",  # Red
    "#FF7F00",  # Orange
    "#FFFF00",  # Yellow
    "#00FF00",  # Green
    "#0000FF",  # Blue
    "#4B0082",  # Indigo
    "#8A2BE2",  # Violet
    "#FF1493",  # Pink
    "#00FFFF",  # Cyan
    "#FF00FF",  # Magenta
    "#FFBF00",  # Amber
    "#008080",  # Teal
    "#7FFF00",  # Chartreuse
    "#E6E6FA",  # Lavender
    "#800000",  # Maroon
    "#FFD700",  # Gold
    "#808000"   # Olive
]


# You can access the hex codes using red_shades[index]


fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, sharex=True, figsize=[10, 8])

He_T_index = 0
He_n_index = 2
He_RV_index = 5
He_EW_index = -2

for k, day in enumerate(fit_params_dict.keys()):
        
    date = date_dict[day]

    #Base points
    ax1.errorbar(date, fit_params_dict[day][:, He_T_index], yerr=fit_err_dict[day][:, He_T_index], fmt='.', color = 'darkgray', alpha=0.01, zorder=-1)
    ax2.errorbar(date, fit_params_dict[day][:, He_n_index], yerr=fit_err_dict[day][:, He_n_index], fmt='.', color = 'darkgray', alpha=0.01, zorder=-1)
    ax3.errorbar(date, fit_params_dict[day][:, He_RV_index], yerr=fit_err_dict[day][:, He_RV_index], fmt='.', color = 'darkgray', alpha=0.01, zorder=-1)
    ax4.errorbar(date, fit_params_dict[day][:, He_EW_index], yerr=fit_err_dict[day][:, He_EW_index], fmt='.', color = 'darkgray', alpha=0.01, zorder=-1)

    ax1.plot(date, fit_params_dict[day][:, He_T_index], '.', color = 'darkgray', alpha=0.1)
    ax2.plot(date, fit_params_dict[day][:, He_n_index], '.', color = 'darkgray', alpha=0.1)
    ax3.plot(date, fit_params_dict[day][:, He_RV_index],  '.', color = 'darkgray', alpha=0.1)
    ax4.plot(date, fit_params_dict[day][:, He_EW_index],  '.', color = 'darkgray', alpha=0.1)

    #Binned points - 1 point for each day
    average_T = np.average(fit_params_dict[day][:, He_T_index], weights = (1/fit_err_dict[day][:, He_T_index])**2)
    average_n = np.average(fit_params_dict[day][:, He_n_index], weights = (1/fit_err_dict[day][:, He_n_index])**2)
    average_RV = np.average(fit_params_dict[day][:, He_RV_index], weights = (1/fit_err_dict[day][:, He_RV_index])**2)
    average_EW = np.average(fit_params_dict[day][:, He_EW_index], weights = (1/fit_err_dict[day][:, He_EW_index])**2)

    ax1.plot(np.average(date), average_T, marker='.', color=day_color[k], markersize=10, label=day)
    ax2.plot(np.average(date), average_n, marker='.', color=day_color[k], markersize=10, label=day)
    ax3.plot(np.average(date), average_RV, marker='.', color=day_color[k], markersize=10, label=day)
    ax4.plot(np.average(date), average_EW, marker='.', color=day_color[k], markersize=10, label=day)
    

ax4.set_xlabel('Time (BJD)')
ax1.set_ylabel('Temperature (K)')
ax2.set_ylabel('Column Density (m$^{-2}$)')
ax3.set_ylabel('Position (m/s)')
ax4.set_ylabel('Equivalent Width ($\AA$)')
plt.legend(bbox_to_anchor=(1, 3.035), fontsize=12)
fig.subplots_adjust(hspace=0)
#plt.savefig(main_directory+'/Big-Exclusion-Main-Plot.pdf')
ax1.set_title('He Triplet Properties Over A Month')
plt.show()
    
    

## Main Plot + Periodogram

In [ ]:
He_T_index = 0
He_n_index = 2
He_RV_index = 5
He_EW_index = -2

probas = [0.1, 0.01, 0.001]

total_date = []
total_T = []
total_n = []
total_RV = []
total_EW = []
total_T_err = []
total_n_err = []
total_RV_err = []
total_EW_err = []

for day in fit_params_dict.keys():
    date = date_dict[day]
    
    #Adding to the big lists
    total_date.append(date)
    total_T.append(fit_params_dict[day][:, He_T_index])
    total_n.append(fit_params_dict[day][:, He_n_index])
    total_RV.append(fit_params_dict[day][:, He_RV_index])
    total_EW.append(fit_params_dict[day][:, He_EW_index])

    total_T_err.append(fit_err_dict[day][:, He_T_index])
    total_n_err.append(fit_err_dict[day][:, He_n_index])
    total_RV_err.append(fit_err_dict[day][:, He_RV_index])
    total_EW_err.append(fit_err_dict[day][:, He_EW_index])

total_date = np.array(list(itertools.chain.from_iterable(total_date)))
total_T = np.array(list(itertools.chain.from_iterable(total_T)))
total_n = np.array(list(itertools.chain.from_iterable(total_n)))
total_RV = np.array(list(itertools.chain.from_iterable(total_RV)))
total_EW = np.array(list(itertools.chain.from_iterable(total_EW)))
total_T_err = np.array(list(itertools.chain.from_iterable(total_T_err)))
total_n_err = np.array(list(itertools.chain.from_iterable(total_n_err)))
total_RV_err = np.array(list(itertools.chain.from_iterable(total_RV_err)))
total_EW_err = np.array(list(itertools.chain.from_iterable(total_EW_err)))

LS_T = LombScargle(total_date, total_T, total_T_err).autopower(nyquist_factor=1)
FAP_T = LombScargle(total_date, total_T, total_T_err).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  

LS_n = LombScargle(total_date, total_n, total_n_err).autopower(nyquist_factor=1)
FAP_n = LombScargle(total_date, total_n, total_n_err).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  

LS_RV = LombScargle(total_date, total_RV, total_RV_err).autopower(nyquist_factor=1)
FAP_RV = LombScargle(total_date, total_RV, total_RV_err).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  

LS_EW = LombScargle(total_date, total_EW, total_EW_err).autopower(nyquist_factor=1)
FAP_EW = LombScargle(total_date, total_EW, total_EW_err).false_alarm_level(probas, method='bootstrap', nyquist_factor=1)  


In [ ]:
reload(AF)

# List of red shades
day_color = [
    "#FF0000",  # Red
    "#FF7F00",  # Orange
    "#FFFF00",  # Yellow
    "#00FF00",  # Green
    "#0000FF",  # Blue
    "#4B0082",  # Indigo
    "#8A2BE2",  # Violet
    "#FF1493",  # Pink
    "#00FFFF",  # Cyan
    "#FF00FF",  # Magenta
    "#FFBF00",  # Amber
    "#008080",  # Teal
    "#7FFF00",  # Chartreuse
    "#E6E6FA",  # Lavender
    "#800000",  # Maroon
    "#FFD700",  # Gold
    "#808000"   # Olive
]

fig, ((ax1, ax5), (ax2, ax6), (ax3, ax7), (ax4, ax8)) = plt.subplots(4, 2, figsize=[10, 8], width_ratios=(4, 1))

He_T_index = 0
He_n_index = 2
He_RV_index = 5
He_EW_index = -2

for k, day in enumerate(fit_params_dict.keys()):
        
    date = date_dict[day]

    #Base points
    ax1.errorbar(date, fit_params_dict[day][:, He_T_index], yerr=fit_err_dict[day][:, He_T_index], fmt='.', color = 'darkgray', alpha=0.01, zorder=-1)
    ax2.errorbar(date, fit_params_dict[day][:, He_n_index], yerr=fit_err_dict[day][:, He_n_index], fmt='.', color = 'darkgray', alpha=0.01, zorder=-1)
    ax3.errorbar(date, fit_params_dict[day][:, He_RV_index], yerr=fit_err_dict[day][:, He_RV_index], fmt='.', color = 'darkgray', alpha=0.01, zorder=-1)
    ax4.errorbar(date, fit_params_dict[day][:, He_EW_index], yerr=fit_err_dict[day][:, He_EW_index], fmt='.', color = 'darkgray', alpha=0.01, zorder=-1)

    ax1.plot(date, fit_params_dict[day][:, He_T_index], '.', color = 'darkgray', alpha=0.1)
    ax2.plot(date, fit_params_dict[day][:, He_n_index], '.', color = 'darkgray', alpha=0.1)
    ax3.plot(date, fit_params_dict[day][:, He_RV_index],  '.', color = 'darkgray', alpha=0.1)
    ax4.plot(date, fit_params_dict[day][:, He_EW_index],  '.', color = 'darkgray', alpha=0.1)

    #Binned points - 1 point for each day
    average_T = np.average(fit_params_dict[day][:, He_T_index], weights = (1/fit_err_dict[day][:, He_T_index])**2)
    average_n = np.average(fit_params_dict[day][:, He_n_index], weights = (1/fit_err_dict[day][:, He_n_index])**2)
    average_RV = np.average(fit_params_dict[day][:, He_RV_index], weights = (1/fit_err_dict[day][:, He_RV_index])**2)
    average_EW = np.average(fit_params_dict[day][:, He_EW_index], weights = (1/fit_err_dict[day][:, He_EW_index])**2)

    ax1.plot(np.average(date), average_T, marker='.', color=day_color[k], markersize=10, label=day)
    ax2.plot(np.average(date), average_n, marker='.', color=day_color[k], markersize=10, label=day)
    ax3.plot(np.average(date), average_RV, marker='.', color=day_color[k], markersize=10, label=day)
    ax4.plot(np.average(date), average_EW, marker='.', color=day_color[k], markersize=10, label=day)
        

ax5.loglog(LS_T[0], LS_T[1], linewidth=1.2)
ax6.loglog(LS_n[0], LS_n[1], linewidth=1.2)
ax7.loglog(LS_RV[0], LS_RV[1], linewidth=1.2)
ax8.loglog(LS_EW[0], LS_EW[1], linewidth=1.2)

#ax5.yaxis.tick_right()
#ax6.yaxis.tick_right()
#ax7.yaxis.tick_right()
#ax8.yaxis.tick_right()

FAPs = [FAP_T, FAP_n, FAP_RV, FAP_EW]
for i, ax in enumerate([ax5, ax6, ax7, ax8]):
    ax.axhline(FAPs[i][0], color='k', linewidth=1, label='F.A.P. = '+str(int(100*probas[0]))+'%', alpha=0.7)
    ax.axhline(FAPs[i][1], color='k', linestyle='-.', linewidth=1, label='F.A.P. = '+str(int(100*probas[1]))+'%', alpha=0.7)
    ax.axhline(FAPs[i][2], color='k', linestyle='--', linewidth=1, label='F.A.P. = '+str(100*probas[2])+'%', alpha=0.7)
    ax.axvline(1/27, color='r', label='Rotation')
    #ax.axvline(2/27, color='orange', label='Rotation, $1^{st}$ Harmonic')
    #ax.axvline(3/27, color='yellow', label='Rotation, $2^{nd}$ Harmonic')


ax4.set_xlabel('Time (BJD)')
ax1.set_ylabel('Temperature (K)')
ax2.set_ylabel('Column Density (m$^{-2}$)')
ax3.set_ylabel('Position (m/s)')
ax4.set_ylabel('Equivalent Width ($\AA$)')
ax1.legend(bbox_to_anchor=(1.63, 1))
ax5.legend(bbox_to_anchor=(2.05, -1.5))
fig.subplots_adjust(wspace = 0.15, hspace=0)
ax1.set_title('He Triplet Properties Over A Month')
#plt.savefig(main_directory+'/Big-Exclusion-Main-Plot-Periodogram.pdf')
plt.show()
    
    

## Main Plot + Activity Index

In [ ]:
#Extracting indices
HARPS_file = np.loadtxt(main_directory+'/SUN_harps.rdb', skiprows=2)

BJD_date = HARPS_file[:, 0]
old_rhk = HARPS_file[:, 9]
old_rhk_err = HARPS_file[:, 10]

low_rhk = -5.02
high_rhk = -4.95

rhk_BJD_date = AF.bound(low_rhk, high_rhk, old_rhk, BJD_date)
rhk = AF.bound(low_rhk, high_rhk, old_rhk, old_rhk)
rhk_err = AF.bound(low_rhk, high_rhk, old_rhk, old_rhk_err)

In [ ]:
#Only taking values at good times
cut_value = 59940

cut_rhk_BJD_date = rhk_BJD_date[rhk_BJD_date>cut_value]
cut_rhk = rhk[rhk_BJD_date>cut_value]
cut_rhk_err = rhk_err[rhk_BJD_date>cut_value]


#Only taking values at best airmass conditions

cut_down = 0.15
cut_up = 0.25
#Find the start dates for each day
integer_dates = np.unique(np.around(cut_rhk_BJD_date))

best_rhk = []
best_date = []
best_rhk_err = []

#Keep the values only in a certain range
for i in range(len(cut_rhk_BJD_date)):
    for j in range(len(integer_dates)):
        if cut_rhk_BJD_date[i]<integer_dates[j]+cut_up and cut_rhk_BJD_date[i]>integer_dates[j]+cut_down:
            best_date.append(cut_rhk_BJD_date[i])
            best_rhk.append(cut_rhk[i])
            best_rhk_err.append(cut_rhk_err[i])


best_date = np.array(best_date)
best_rhk = np.array(best_rhk)
best_rhk_err = np.array(best_rhk_err)

In [ ]:
#Bin the values to one value for each day

#Find the start dates for each day
integer_dates = np.unique(np.around(best_date))
bin_best_rhk = []
bin_best_date = []
bin_best_rhk_err = []

for j in range(len(integer_dates)):
    store_time = []
    store_rhk = []
    store_rhk_err = []
    
    for i in range(len(best_date)):
        if best_date[i] >= integer_dates[j-1] and best_date[i] < integer_dates[j]:
            store_time.append(best_date[i])
            store_rhk.append(best_rhk[i])
            store_rhk_err.append(best_rhk_err[i])
    #Skipping first iteration because it is an empty list
    if j>=1:
        bin_best_date.append(np.mean(store_time))
        bin_best_rhk.append(np.mean(store_rhk))
        bin_best_rhk_err.append(np.mean(store_rhk_err))


In [ ]:
## Fit activity index with basic model
#Perform a periodogram to get the period of variations for Rhk and Smw
rhk_periodo = LombScargle(bin_best_date, bin_best_rhk, bin_best_rhk_err).autopower(nyquist_factor=1)

max_power = np.where(rhk_periodo[1]==max(rhk_periodo[1]))

period_rhk = 1/rhk_periodo[0][max_power][0]
period_rhk_2 = period_rhk/2

def big_model_rhk(t, offset, A, B, C, D):
    
    period = period_rhk
    period2 = period_rhk_2
    
    first_part = A*np.cos(2*np.pi*t/period) + B*np.sin(2*np.pi*t/period)
    second_part = C*np.cos(2*np.pi*t/period2) + D*np.sin(2*np.pi*t/period2)
    
    return offset+first_part+second_part

best_fit_theta_rhk = curve_fit(big_model_rhk, bin_best_date, bin_best_rhk, sigma=bin_best_rhk_err)[0]

#Simulating best-fit model points
model_time = np.linspace(min(bin_best_date), max(bin_best_date), 1000)
model_data_rhk = big_model_rhk(model_time, *best_fit_theta_rhk)

In [ ]:
reload(AF)

# List of red shades
day_color = [
    "#FF0000",  # Red
    "#FF7F00",  # Orange
    "#FFFF00",  # Yellow
    "#00FF00",  # Green
    "#0000FF",  # Blue
    "#4B0082",  # Indigo
    "#8A2BE2",  # Violet
    "#FF1493",  # Pink
    "#00FFFF",  # Cyan
    "#FF00FF",  # Magenta
    "#FFBF00",  # Amber
    "#008080",  # Teal
    "#7FFF00",  # Chartreuse
    "#E6E6FA",  # Lavender
    "#800000",  # Maroon
    "#FFD700",  # Gold
    "#808000"   # Olive
]

fig, ((ax1, ax6), (ax2, ax7), (ax3, ax8), (ax4, ax9), (ax5, ax10)) = plt.subplots(5, 2, figsize=[10, 8], width_ratios=(4, 1))

He_T_index = 0
He_n_index = 2
He_RV_index = 5
He_EW_index = -2

averages_T = []
averages_n = []
averages_RV = []
averages_EW = []
averages_RHK = []
total_RHK = []
for k, day in enumerate(fit_params_dict.keys()):
        
    date = date_dict[day]
    
    plot_T = 10**fit_params_dict[day][:, He_T_index]
    plot_T_err = np.log(10)* fit_err_dict[day][:, He_T_index] * 10**fit_params_dict[day][:, He_T_index]
    #Base points
    ax1.errorbar(date, plot_T, yerr=plot_T_err, fmt='.', color = 'darkgray', alpha=0.01, zorder=-1)
    ax2.errorbar(date, fit_params_dict[day][:, He_n_index], yerr=fit_err_dict[day][:, He_n_index], fmt='.', color = 'darkgray', alpha=0.01, zorder=-1)
    ax3.errorbar(date, fit_params_dict[day][:, He_RV_index], yerr=fit_err_dict[day][:, He_RV_index], fmt='.', color = 'darkgray', alpha=0.01, zorder=-1)
    ax4.errorbar(date, fit_params_dict[day][:, He_EW_index], yerr=fit_err_dict[day][:, He_EW_index], fmt='.', color = 'darkgray', alpha=0.01, zorder=-1)
    ax5.plot(date, big_model_rhk(date, *best_fit_theta_rhk), marker='.', color = 'darkgray', zorder=-1)
    
    ax1.plot(date, plot_T, '.', color = 'darkgray', alpha=0.1)
    ax2.plot(date, fit_params_dict[day][:, He_n_index], '.', color = 'darkgray', alpha=0.1)
    ax3.plot(date, fit_params_dict[day][:, He_RV_index],  '.', color = 'darkgray', alpha=0.1)
    ax4.plot(date, fit_params_dict[day][:, He_EW_index],  '.', color = 'darkgray', alpha=0.1)

    #Binned points - 1 point for each day
    average_T = np.average(plot_T, weights = (1/plot_T_err)**2)
    average_n = np.average(fit_params_dict[day][:, He_n_index], weights = (1/fit_err_dict[day][:, He_n_index])**2)
    average_RV = np.average(fit_params_dict[day][:, He_RV_index], weights = (1/fit_err_dict[day][:, He_RV_index])**2)
    average_EW = np.average(fit_params_dict[day][:, He_EW_index], weights = (1/fit_err_dict[day][:, He_EW_index])**2)

    ax1.plot(np.average(date), average_T, marker='.', color=day_color[k], markersize=8, label=day[5:])
    ax2.plot(np.average(date), average_n, marker='.', color=day_color[k], markersize=8, label=day[5:])
    ax3.plot(np.average(date), average_RV, marker='.', color=day_color[k], markersize=8, label=day[5:])
    ax4.plot(np.average(date), average_EW, marker='.', color=day_color[k], markersize=8, label=day[5:])
    ax5.plot(np.average(date), big_model_rhk(np.average(date), *best_fit_theta_rhk), marker='.', color=day_color[k], markersize=8, label=day[5:])

    ax6.plot(big_model_rhk(np.average(date), *best_fit_theta_rhk), average_T, '.')
    ax7.plot(big_model_rhk(np.average(date), *best_fit_theta_rhk), average_n, '.')
    ax8.plot(big_model_rhk(np.average(date), *best_fit_theta_rhk), average_RV, '.')
    ax9.plot(big_model_rhk(np.average(date), *best_fit_theta_rhk), average_EW, '.')
    ax10.plot(big_model_rhk(np.average(date), *best_fit_theta_rhk), big_model_rhk(np.average(date), *best_fit_theta_rhk), '.')
    
    averages_T.append(average_T)
    averages_n.append(average_n)
    averages_RV.append(average_RV)
    averages_EW.append(average_EW)
    averages_RHK.append(big_model_rhk(np.average(date), *best_fit_theta_rhk))
    
ax6.axes.get_yaxis().set_visible(False)
ax7.axes.get_yaxis().set_visible(False)
ax8.axes.get_yaxis().set_visible(False)
ax9.axes.get_yaxis().set_visible(False)
ax10.axes.get_yaxis().set_visible(False)
ax10.axes.get_xaxis().set_visible(False)


textstr = (r"$r_P = %.3f$" % (np.corrcoef(averages_T, averages_RHK)[0][1], ))
ax6.text(0.4, 0.1, textstr, transform=ax6.transAxes, fontsize=10, bbox = dict(facecolor='white', alpha=0.5))

textstr = (r"$r_P = %.3f$" % (np.corrcoef(averages_n, averages_RHK)[0][1], ))
ax7.text(0.4, 0.1, textstr, transform=ax7.transAxes, fontsize=10, bbox = dict(facecolor='white', alpha=0.5))

textstr = (r"$r_P = %.3f$" % (np.corrcoef(averages_RV, averages_RHK)[0][1], ))
ax8.text(0.4, 0.1, textstr, transform=ax8.transAxes, fontsize=10, bbox = dict(facecolor='white', alpha=0.5))

textstr = (r"$r_P = %.3f$" % (np.corrcoef(averages_EW, averages_RHK)[0][1], ))
ax9.text(0.4, 0.1, textstr, transform=ax9.transAxes, fontsize=10, bbox = dict(facecolor='white', alpha=0.5))

textstr = (r"$r_P = %.3f$" % (np.corrcoef(averages_RHK, averages_RHK)[0][1], ))
ax10.text(0.4, 0.1, textstr, transform=ax10.transAxes, fontsize=10, bbox = dict(facecolor='white', alpha=0.5))

ax5.set_xlabel('Time (BJD)')
ax1.set_ylabel('Temp. (K)')
ax2.set_ylabel('Log_{10} Col. Density (cm$^{-2}$)')
ax3.set_ylabel('Position (m/s)')
ax4.set_ylabel('Eq. Width ($\AA$)')
ax5.set_ylabel('$R_{HK}$')
ax1.legend(ncol=7, bbox_to_anchor=(1.52, 1))
#ax6.set_title('$R_{HK}$ Correlation')
fig.subplots_adjust(wspace = 0, hspace=0)
#ax1.set_title('He Triplet Properties Over A Month')
plt.savefig(main_directory+'/Big-Exclusion-Main-Plot-RHK.pdf')
plt.show()
    
